# Applied Dash Plotly to visualize Covid-19 Dashboard

## Import Library

In [2]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

## Download data

In [3]:
url_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
url_recovered = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

confirmed = pd.read_csv(url_confirmed)
deaths = pd.read_csv(url_deaths)
recovered = pd.read_csv(url_recovered)

In [4]:
confirmed.sample(10)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
76,Jiangxi,China,27.614000,115.722100,2,7,18,18,36,72,...,3423,3423,3423,3423,3423,3423,3423,3423,3423,3423
169,NaN,Lesotho,-29.610000,28.233600,0,0,0,0,0,0,...,34790,34790,34790,34790,34790,34790,34790,34790,34790,34790
99,NaN,Croatia,45.100000,15.200000,0,0,0,0,0,0,...,1268992,1268992,1268992,1268992,1268992,1268992,1269326,1269326,1269326,1269326
205,NaN,Niger,17.607789,8.081666,0,0,0,0,0,0,...,9508,9508,9508,9508,9508,9508,9508,9508,9508,9508
151,NaN,Iraq,33.223191,43.679291,0,0,0,0,0,0,...,2465545,2465545,2465545,2465545,2465545,2465545,2465545,2465545,2465545,2465545
123,Martinique,France,14.641500,-61.024200,0,0,0,0,0,0,...,228875,228875,228875,228875,228875,228875,228875,228875,229020,229020
158,NaN,Kazakhstan,48.019600,66.923700,0,0,0,0,0,0,...,1498668,1498668,1498668,1498668,1498668,1498668,1498668,1498668,1498668,1498668
229,NaN,Saudi Arabia,23.885942,45.079162,0,0,0,0,0,0,...,829478,829554,829624,829624,829730,829784,829882,829955,830044,830127
106,NaN,Diamond Princess,0.000000,0.000000,0,0,0,0,0,0,...,712,712,712,712,712,712,712,712,712,712
273,Jersey,United Kingdom,49.213800,-2.135800,0,0,0,0,0,0,...,66391,66391,66391,66391,66391,66391,66391,66391,66391,66391


## Preprocessing Data

### Melting data

In [5]:
confirmed_date = confirmed.columns[4:] 
confirmed_geo = confirmed.columns[:4]
total_confirmed = confirmed.melt(id_vars=confirmed_geo, value_vars= confirmed_date,value_name='Confirmed',var_name='Date')
# total_confirmed
deaths_date = deaths.columns[4:]
deaths_geo = deaths.columns[:4]
total_deaths = deaths.melt(id_vars=confirmed_geo, value_vars=deaths_date, value_name='Deaths',var_name='Date')

recovered_date = recovered.columns[4:]
recovered_geo = recovered.columns[:4]
total_recovered= recovered.melt(id_vars=recovered_geo, value_vars=recovered_date, value_name='Recovered',var_name='Date')

### Merging Data

In [6]:
covid_data = total_confirmed.merge(total_deaths, on=confirmed_geo.to_list() + ['Date'], how='left')
covid_data = covid_data.merge(total_recovered, on = recovered_geo.to_list() + ['Date'], how='left')

In [7]:
covid_data.sample(10)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
191067,NaN,Cambodia,11.550000,104.91670,11/13/21,119481,2861,0.0
6293,NaN,Saint Lucia,13.909400,-60.97890,2/12/20,0,0,0.0
212690,Pitcairn Islands,United Kingdom,-24.376800,-128.32420,1/26/22,0,0,0.0
306088,NaN,Cabo Verde,16.538800,-23.04180,12/16/22,63159,412,0.0
295883,NaN,Slovenia,46.151200,14.99550,11/10/22,1240891,6908,0.0
120010,Jiangsu,China,32.971100,119.45500,3/12/21,705,0,704.0
190375,NaN,Papua New Guinea,-6.314993,143.95555,11/10/21,32279,415,0.0
83091,NaN,India,20.593684,78.96288,11/4/20,8364086,124315,7711809.0
160240,NaN,Georgia,42.315400,43.35690,7/29/21,413626,5768,379685.0
153040,NaN,Kenya,-0.023600,37.90620,7/4/21,185868,3675,127625.0


In [8]:
covid_data['Date'] = pd.to_datetime(covid_data['Date'])

/tmp/ipykernel_12497/3902955790.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  covid_data['Date'] = pd.to_datetime(covid_data['Date'])


In [9]:
covid_data.isna().sum()

Province/State    226314
Country/Region         0
Lat                 2286
Long                2286
Date                   0
Confirmed              0
Deaths                 0
Recovered          24003
dtype: int64

In [10]:
covid_data['Recovered'] = covid_data['Recovered'].fillna(0)

In [11]:
covid_data['Active'] = covid_data['Confirmed'] - covid_data['Deaths'] - covid_data['Recovered']

In [12]:
covid_data_1 = covid_data.groupby(['Date'])[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum().reset_index()
covid_data_1

,Date,Confirmed,Deaths,Recovered,Active
0,2020-01-22,557,17,30.0,510.0
1,2020-01-23,657,18,32.0,607.0
2,2020-01-24,944,26,39.0,879.0
3,2020-01-25,1437,42,42.0,1353.0
4,2020-01-26,2120,56,56.0,2008.0
...,...,...,...,...,...
1138,2023-03-05,676024901,6877749,0.0,669147152.0
1139,2023-03-06,676082941,6878115,0.0,669204826.0
1140,2023-03-07,676213378,6879038,0.0,669334340.0
1141,2023-03-08,676392824,6880483,0.0,669512341.0


In [13]:
covid_data_2 = covid_data.groupby(['Date', 'Country/Region'])[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum().reset_index()
covid_data_2

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
229738,2023-03-09,West Bank and Gaza,703228,5708,0.0,697520.0
229739,2023-03-09,Winter Olympics 2022,535,0,0.0,535.0
229740,2023-03-09,Yemen,11945,2159,0.0,9786.0
229741,2023-03-09,Zambia,343135,4057,0.0,339078.0


In [14]:
covid_data_dict = covid_data[['Country/Region', 'Lat', 'Long']]
list_locations = covid_data_dict.set_index('Country/Region')[['Lat', 'Long']].T.to_dict()

/tmp/ipykernel_12497/3770569221.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  list_locations = covid_data_dict.set_index('Country/Region')[['Lat', 'Long']].T.to_dict()


In [19]:
print(covid_data['Date'])

0        2020-01-22
1        2020-01-22
2        2020-01-22
3        2020-01-22
4        2020-01-22
            ...    
330322   2023-03-09
330323   2023-03-09
330324   2023-03-09
330325   2023-03-09
330326   2023-03-09
Name: Date, Length: 330327, dtype: datetime64[ns]


## Creating Dashboard

Continue at main.py

In [16]:
# app = dash.Dash(__name__, meta_tags=[{"name": "viewport", "content": "width=device-width"}])

In [17]:
# app.layout = html.Div([
#     html.Div([
#         html.Div([
#             html.Img(src=app.get_asset_url('OIP.jpeg'),
#                      id='corona-image',
#                      style={
#                          'height':'60px',
#                          'width': 'auto',
#                          'margin-bottom':'25px',
#                      })
#         ])
#     ])
# ])

In [18]:
# if __name__ == '__main__':
#     app.run_server()